In [ ]:
import pandas as pd

In [ ]:
path_external = "/content/drive/MyDrive/CoVerifi&MedVerifi/" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_external+"Combined_News.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.8 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 31.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.9 MB/s 
     |████████████████████████████████| 140 kB 64.6 MB/s 
     |████████████████████████████████| 212 kB 67.3 MB/s 
     |████████████████████████████████| 1.1 MB 59.3 MB/s 
     |████████████████████████████████| 127 kB 76.5 MB/s 
     |████████████████████████████████| 144 kB 59.4 MB/s 
     |████████████████████████████████| 271 kB 64.5 MB/s 
     |█████████████

In [ ]:
model_path = '/content/drive/MyDrive/CoVerifi&MedVerifi/CoVerifi/CoVerifi/ML_Model/'

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import AutoModelForSequenceClassification
import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
import numpy as np
from datasets import load_metric
from transformers import Trainer
from transformers import TrainingArguments

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)


# Validate on BERT

## Trained on FNN&CoAID

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

import pandas as pd
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer("Attention is all you need")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [ ]:
import numpy as np
from datasets import load_metric
from transformers import Trainer
from transformers import TrainingArguments

#Load saved model
import torch
model.load_state_dict(torch.load(model_path+'finetune_bert_on_CoAID_and_all_of_FNN.pt'))

#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.8262717868782768,
 'eval_f1': 0.7688536953242836,
 'eval_loss': 0.7342257499694824,
 'eval_precision': 0.9572769953051643,
 'eval_recall': 0.6424070573408948,
 'eval_runtime': 132.534,
 'eval_samples_per_second': 53.247,
 'eval_steps_per_second': 6.662}

## Out of box

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_accuracy': 0.5502338104010203,
 'eval_f1': 0.0,
 'eval_loss': 0.7079910635948181,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 131.6827,
 'eval_samples_per_second': 53.591,
 'eval_steps_per_second': 6.706}

## finetune_bert_on_CoAID&GossipCop_only

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID&GossipCop_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.8136601955505172,
 'eval_f1': 0.7464815885868517,
 'eval_loss': 0.9527012705802917,
 'eval_precision': 0.9617486338797814,
 'eval_recall': 0.6099558916194077,
 'eval_runtime': 131.5944,
 'eval_samples_per_second': 53.627,
 'eval_steps_per_second': 6.71}

## CoAID 

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.9223466062066034,
 'eval_f1': 0.9073385187690227,
 'eval_loss': 0.4804149270057678,
 'eval_precision': 0.9791970802919708,
 'eval_recall': 0.8453056080655325,
 'eval_runtime': 131.5581,
 'eval_samples_per_second': 53.642,
 'eval_steps_per_second': 6.712}

## finetune_bert_on_CoAID&PolitiFact_only

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_CoAID&PolitiFact_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.7992064616692646,
 'eval_f1': 0.7162026837572602,
 'eval_loss': 1.113200306892395,
 'eval_precision': 0.982957669048928,
 'eval_recall': 0.5633270321361059,
 'eval_runtime': 131.7258,
 'eval_samples_per_second': 53.573,
 'eval_steps_per_second': 6.703}

## finetune_bert_on_all_of_FNN.pt

In [ ]:
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_all_of_FNN.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.5359217797931132,
 'eval_f1': 0.5065541660388729,
 'eval_loss': 1.7433199882507324,
 'eval_precision': 0.48541726826451054,
 'eval_recall': 0.5296156269691241,
 'eval_runtime': 131.5365,
 'eval_samples_per_second': 53.65,
 'eval_steps_per_second': 6.713}

## finetune_bert_on_GossipCop_only.pt

In [ ]:
import torch
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_GossipCop_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.3661612583250673,
 'eval_f1': 0.3895182202811519,
 'eval_loss': 2.2356884479522705,
 'eval_precision': 0.34360703106188295,
 'eval_recall': 0.4495904221802142,
 'eval_runtime': 131.5074,
 'eval_samples_per_second': 53.662,
 'eval_steps_per_second': 6.714}

##finetune_bert_on_PolitiFact_only

In [ ]:

import torch
#Load model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

#Load saved model
model_name = 'finetune_bert_on_PolitiFact_only.pt'
model.load_state_dict(torch.load(model_path+model_name))

trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

{'eval_accuracy': 0.7092248830947995,
 'eval_f1': 0.5672711935892029,
 'eval_loss': 1.2808611392974854,
 'eval_precision': 0.857780612244898,
 'eval_recall': 0.423755513547574,
 'eval_runtime': 131.7779,
 'eval_samples_per_second': 53.552,
 'eval_steps_per_second': 6.701}

# Roberta-Fake-News

## 1.Out-of-box

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp8dijqi3_


Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

storing https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/0dc2cdacd2fe1dd988f8f9976e21db42274012f2d86da28d573081dbff2a059b.71056ac4e5c419bd472fc1a3f4269ad35695eb83fe663f969180490939da9f9c
creating metadata file for /root/.cache/huggingface/transformers/0dc2cdacd2fe1dd988f8f9976e21db42274012f2d86da28d573081dbff2a059b.71056ac4e5c419bd472fc1a3f4269ad35695eb83fe663f969180490939da9f9c
https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplgup_83y


Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

storing https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
creating metadata file for /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
 

...

storing https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/5d86bda5dc16a19ae0ebc44499ec825b06516f9aae277aff35d2ca814da2a5f1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/5d86bda5dc16a19ae0ebc44499ec825b06516f9aae277aff35d2ca814da2a5f1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplf2wd5oa


Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

storing https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/424977d8969812c880aa638f4451597de6d4c1c429d98b7aed0822f8c327bbcb.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
creating metadata file for /root/.cache/huggingface/transformers/424977d8969812c880aa638f4451597de6d4c1c429d98b7aed0822f8c327bbcb.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
loading file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/d4210e7379aab0e68db3bb082412960f0b90193184d6938abedc2212cffc8ee9.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/5d86bda5dc16a19ae0ebc44499ec825b06516f9aae277aff35d2ca814da2a5f1.5d12962c5ee615a4c803841266e9c3be9

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

storing https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/25df84fde28d696e85355be437506a664875ce39baf69ad4d961f1ae820d5c7c.6bd0b65f7890a98046e2f7a40bdf15ee81164fd1e56d307d16599a4e9af5250a
creating metadata file for /root/.cache/huggingface/transformers/25df84fde28d696e85355be437506a664875ce39baf69ad4d961f1ae820d5c7c.6bd0b65f7890a98046e2f7a40bdf15ee81164fd1e56d307d16599a4e9af5250a
loading weights file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/25df84fde28d696e85355be437506a664875ce39baf69ad4d961f1ae820d5c7c.6bd0b65f7890a98046e2f7a40bdf15ee81164fd1e56d307d16599a4e9af5250a
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at ghanashyamvtatti/roberta-fake-news.
If your ta

In [ ]:
#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.4545841008927306,
 'eval_f1': 0.6212732460887533,
 'eval_loss': 1.2726035118103027,
 'eval_precision': 0.4517098297324367,
 'eval_recall': 0.9946439823566477,
 'eval_runtime': 130.6377,
 'eval_samples_per_second': 54.02,
 'eval_steps_per_second': 6.759}

## 2.Finetuned on CoAID 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_with_shuffle.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()


loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.9562136885362051,
 'eval_f1': 0.9496660693924092,
 'eval_loss': 0.2911560833454132,
 'eval_precision': 0.9831365935919055,
 'eval_recall': 0.9183994959042218,
 'eval_runtime': 130.4395,
 'eval_samples_per_second': 54.102,
 'eval_steps_per_second': 6.769}

## 3.Fine-tuned on FNN

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.5235935950120448,
 'eval_f1': 0.5731335703402742,
 'eval_loss': 1.0985732078552246,
 'eval_precision': 0.48000850701829006,
 'eval_recall': 0.7110901071203529,
 'eval_runtime': 130.3896,
 'eval_samples_per_second': 54.122,
 'eval_steps_per_second': 6.772}

## 4.Fine-tuned on CoAID&FNN

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.8153606348306646,
 'eval_f1': 0.7556722295143447,
 'eval_loss': 0.5768745541572571,
 'eval_precision': 0.9333024548402038,
 'eval_recall': 0.6348456206679269,
 'eval_runtime': 130.4397,
 'eval_samples_per_second': 54.102,
 'eval_steps_per_second': 6.769}

## 5.Fine-tuned on CoAID&PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&&PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.910726937792263,
 'eval_f1': 0.8923812777587975,
 'eval_loss': 0.43977585434913635,
 'eval_precision': 0.9746268656716418,
 'eval_recall': 0.8229363579080026,
 'eval_runtime': 130.3216,
 'eval_samples_per_second': 54.151,
 'eval_steps_per_second': 6.776}

## 6.Fine-tuned on CoAID & GossipCop

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_CoAID&GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.8806858438429928,
 'eval_f1': 0.8522288522288521,
 'eval_loss': 0.3343069553375244,
 'eval_precision': 0.9619651347068146,
 'eval_recall': 0.7649653434152489,
 'eval_runtime': 130.4747,
 'eval_samples_per_second': 54.087,
 'eval_steps_per_second': 6.768}

## 7.Fine-tuned on GossipCop

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.4336120164375797,
 'eval_f1': 0.6029601668818914,
 'eval_loss': 1.12228262424469,
 'eval_precision': 0.4403017554040331,
 'eval_recall': 0.9562066792690611,
 'eval_runtime': 130.3712,
 'eval_samples_per_second': 54.13,
 'eval_steps_per_second': 6.773}

## 8.Fine-tuned on PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model.load_state_dict(torch.load(model_path+'roberta-fake-news_on_PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/ghanashyamvtatti/roberta-fake-news/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/4e23f7c43b2fa5d4e963f2e3d66f0813e2f2a74e243d39756ceefbb7fd49e3db.423dc4310dc96e26b1aca95fc8c3086041fe0868cf99993dc9f737c846dca967
Model config RobertaConfig {
  "_name_or_path": "ghanashyamvtatti/roberta-fake-news",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 1,
  "use_cach

{'eval_accuracy': 0.6868357659061924,
 'eval_f1': 0.6813725490196078,
 'eval_loss': 1.3481096029281616,
 'eval_precision': 0.6281233386496544,
 'eval_recall': 0.7444864524259609,
 'eval_runtime': 130.4345,
 'eval_samples_per_second': 54.104,
 'eval_steps_per_second': 6.77}

# Fake-News-Bert-Detect

## 1. Out-of-box

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp1uhjpj1s


Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/1bb953ac793c5ae15e9b5a3e1748d62cad5e9e5557408e989fb5d25f00ee6d0c.d6a300aa289d15142bf7dbad3b3a42b7c1ee1e0c874081ff0108003a2e307a2a
creating metadata file for /root/.cache/huggingface/transformers/1bb953ac793c5ae15e9b5a3e1748d62cad5e9e5557408e989fb5d25f00ee6d0c.d6a300aa289d15142bf7dbad3b3a42b7c1ee1e0c874081ff0108003a2e307a2a
https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjz3sqcnk


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
creating metadata file for /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpgcbaxsz9


...

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/6a4467084bef307312ff1896f1584b5a7f28fa88d2c2c39fa2e4905e6e2bdd2c.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
creating metadata file for /root/.cache/huggingface/transformers/6a4467084bef307312ff1896f1584b5a7f28fa88d2c2c39fa2e4905e6e2bdd2c.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/abccee4c0ea0661b34063ece912c47e6d578f8ab52d66c02ea3387a3b5eb79b9.5d12962c5ee615a4c803841266e9c3be9a691a9

Downloading:   0%|          | 0.00/735 [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
creating metadata file for /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

storing https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
creating metadata file for /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
loading weights file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/6cef7d197f6d86a75be415374a6577454a31076c29da2583b8a441b6f273d6b0.c3091640aebed39d220ea7006739d06210601f14925e4fd5acfc493711b8081d
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at jy46604790/Fake-News-Bert-Detect.
If your task is 

In [ ]:
#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)

#Eval Parameters
metric = load_metric("accuracy")
test_args = TrainingArguments("test_trainer", num_train_epochs=3)

#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.7515941618251382,
 'eval_f1': 0.6443497666869548,
 'eval_loss': 1.211885690689087,
 'eval_precision': 0.9048433048433049,
 'eval_recall': 0.5003150598613737,
 'eval_runtime': 130.4769,
 'eval_samples_per_second': 54.086,
 'eval_steps_per_second': 6.767}

## 2.Finetuned on CoAID

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_shuffle_train_on_all_CoAID.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.9773274762647017,
 'eval_f1': 0.9746915533059158,
 'eval_loss': 0.16331693530082703,
 'eval_precision': 0.9787166454891995,
 'eval_recall': 0.9706994328922496,
 'eval_runtime': 130.5464,
 'eval_samples_per_second': 54.057,
 'eval_steps_per_second': 6.764}

## 3.Fine-tuned on FNN 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_Finetune_on_FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.44976618959897974,
 'eval_f1': 0.6204672075065976,
 'eval_loss': 0.6935955882072449,
 'eval_precision': 0.44976618959897974,
 'eval_recall': 1.0,
 'eval_runtime': 130.4526,
 'eval_samples_per_second': 54.096,
 'eval_steps_per_second': 6.769}

## 4.Fine-tuned on CoAID&FNN 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_Finetune_on_CoAID&FNN.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

## 5.Fine-tuned on CoAID&PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_CoAID&PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.8961315006376648,
 'eval_f1': 0.872366359045795,
 'eval_loss': 0.5453846454620361,
 'eval_precision': 0.9750875827170105,
 'eval_recall': 0.7892249527410208,
 'eval_runtime': 130.1771,
 'eval_samples_per_second': 54.211,
 'eval_steps_per_second': 6.783}

## 6.Fine-tuned on CoAID & GossipCop 

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_CoAID&GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.878418591469463,
 'eval_f1': 0.8520179372197311,
 'eval_loss': 0.30670884251594543,
 'eval_precision': 0.9413109756097561,
 'eval_recall': 0.7781978575929427,
 'eval_runtime': 130.8444,
 'eval_samples_per_second': 53.934,
 'eval_steps_per_second': 6.748}

## 7.Fine-tuned on GossipCop 

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_GossipCop_only.pt'))



def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')
#Dataset
dfTotal_external = dfTotal_external.dropna()
dataset_external = Dataset.from_pandas(dfTotal_external)
dataset_external = dataset_external.map(tokenize_data, batched=True)


loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/5a3b85c1b5bf39e7feabb076838b178aa37c4e860c428b2fbd40ef1972f679a0.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/abccee4c0ea0661b34063ece912c47e6d578f8ab52d66c02ea3387a3b5eb79b9.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/tokenizer.json from cache at None
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/6a4467084bef307312ff1896f1584b5a7f28fa88d2c2c39fa2e4905e6

  0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_GossipCop_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

Parameter 'function'=<function tokenize_data at 0x7fc2ae60e8c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.7515941618251382,
 'eval_f1': 0.6443497666869548,
 'eval_loss': 1.211885690689087,
 'eval_precision': 0.9048433048433049,
 'eval_recall': 0.5003150598613737,
 'eval_runtime': 131.6997,
 'eval_samples_per_second': 53.584,
 'eval_steps_per_second': 6.705}

## 8.Fine-tuned on PolitiFact

In [ ]:
#Load saved model
import torch
model = AutoModelForSequenceClassification.from_pretrained("jy46604790/Fake-News-Bert-Detect")

model.load_state_dict(torch.load(model_path+'Fake-News-Bert-Detect_finetuned_on_PolitiFact_only.pt'))


#Setup Evaluation trainer
trainer_eval = Trainer(
    model=model,
    args=test_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)

#Evaluate without re-train the model
trainer_eval.evaluate()

loading configuration file https://huggingface.co/jy46604790/Fake-News-Bert-Detect/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/296e7242cf9d7626ddf66df9e028de52c54a42ea896a02a35a08150aeb0012dc.b3b49a6f9f8c9e38f7810f9e7f04f890283b5f9dc797e68af7ed71eb2b7e6afd
Model config RobertaConfig {
  "_name_or_path": "jy46604790/Fake-News-Bert-Detect",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.0

{'eval_accuracy': 0.7551367436587785,
 'eval_f1': 0.7055214723926381,
 'eval_loss': 1.1625183820724487,
 'eval_precision': 0.7683741648106904,
 'eval_recall': 0.6521739130434783,
 'eval_runtime': 130.8332,
 'eval_samples_per_second': 53.939,
 'eval_steps_per_second': 6.749}

#XLNet

## Import XLNet tokenizer, used to convert our text into tokens that correspond to XLNet’s vocabulary.

In [ ]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from transformers import AdamW

from tqdm import tqdm, trange
import io
import numpy as np
import matplotlib.pyplot as plt

from unicodedata import name
import pandas as pd
import glob, os
from datasets import Dataset
import numpy as np
from datasets import load_metric

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install sentencepiece 
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 9.7 MB/s 
     |████████████████████████████████| 1.8 MB 55.4 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 9.1 MB 50.8 MB/s 
     |████████████████████████████████| 145 kB 72.9 MB/s 
     |████████████████████████████████| 181 kB 55.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 4.3 MB 36.4 MB/s 
     |████████████████████████████████| 164 kB 65.4 MB/s 
     |████████████████████████████████| 111 kB 59.3 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 232 kB 74.6 MB/s 
     |████████████████████████████████| 133 kB 66.6 MB/s

In [ ]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)


## 1. Out-of-box 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.44976618959897974

## 2.Finetuned on CoAID 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID.pt'))

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

<All keys matched successfully>

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set
model.cuda()
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.9436020972084456

## 3.Fine-tuned on FNN 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_FNN.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.5714893014028624

## 4.Fine-tuned on CoAID&FNN

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_FNN.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.8264134901516225

## 5.Fine-tuned on CoAID&PolitiFact 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_Politi.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.8266968966983137

## 6.Fine-tuned on CoAID & GossipCop 

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_CoAID_Gossip.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.8689244721553068

## 7.Fine-tuned on GossipCop

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_Gossip.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.4160408105427235

## 8.Fine-tuned on PolitiFact

In [ ]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.cuda()
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=2)
model.load_state_dict(torch.load(model_path+'XLNet_on_Politi.pt'))
model.cuda()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
df = dfTotal_external
# Create sentence and label lists
sentences = df.title.values

# We need to add special tokens at the beginning and end of each sentence for XLNet to work properly
sentences = [sentence + " [SEP] [CLS]" for sentence in sentences]
labels = df.label.values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]


MAX_LEN = 128
# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(labels)
  
batch_size = 32  


prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(flat_true_labels, flat_predictions)

0.7065325209012329

# Text-CNN

## 1. Out-of-box 2.Finetuned on CoAID 3.Fine-tuned on FNN 4.Fine-tuned on CoAID&FNN 5.Fine-tuned on CoAID&PolitiFact 6.Fine-tuned on CoAID & GossipCop 7.Fine-tuned on GossipCop 8.Fine-tuned on PolitiFact